# MVD 8. cvičení

## 1. část - Vytvoření dat

Použijte stejný kód z minulého cvičení pro vytvoření dat. Navíc vytvořte jeden větší dataset, ve kterém sjednotíte výstupy různých funkcí (např. make_blobs + make_circles). 

In [6]:
import numpy as np
from sklearn.datasets import make_blobs,make_circles
import pandas as pd

In [38]:

## Small dataset
x_s, y_s = make_blobs(n_samples=100, centers=2, n_features=2)
df1 = pd.DataFrame({'x': x_s[:, 0], 'y': x_s[:, 1], "label": y_s, "kmeans": ["TODO"] * 100})
## big dataset
x1, y1 = make_blobs(n_samples=100, centers=2, n_features=2)
x2, y2 = make_circles(n_samples=(100,100), noise=0.3,factor=0.2)
y = np.concatenate((y1+3,y2))
x = np.concatenate((x1,x2), axis=0)
df2 = pd.DataFrame({'x': x[:, 0], 'y': x[:, 1], "label": y, "kmeans": ["TODO"] * 300})
print(df2)

            x         y  label kmeans
0   -4.816033  9.119368      3   TODO
1   -9.638208  2.397894      4   TODO
2   -6.494391  8.356785      3   TODO
3   -6.528399  8.390397      3   TODO
4   -9.239861  0.793520      4   TODO
..        ...       ...    ...    ...
295 -0.286689  0.224299      1   TODO
296 -0.009165  0.751279      0   TODO
297  0.947033 -0.909323      0   TODO
298 -0.115701  0.080887      1   TODO
299  0.639639  0.617883      0   TODO

[300 rows x 4 columns]


## 2. část - Implementace DBSCAN algoritmu
Dle přednášky implementujte DBSCAN algoritmus.

## 3. část - Vyhodnocení 
Aplikujte Váš DBSCAN na vytvořené datasety. Experimentálně najděte parametry ```Eps``` a ```MinPts```.

## 4. část - Porovnání s výstupem z knihovny

Porovnejte výstup s výstupem z knihovny sklearn. Dokumentaci naleznete [zde](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

In [39]:
from sklearn.cluster import DBSCAN
import plotly.express as px
import plotly.graph_objects as go

In [40]:
df = df2

clustering = DBSCAN(eps=3, min_samples=2).fit(df[["x","y"]])

fig = px.scatter(x=df["x"],y=df["y"], color=[str(x)  for x in df["label"]])
fig.show()

fig = px.scatter(x=df["x"],y=df["y"], color=[str(x)  for x in clustering.labels_])
fig.show()